In [155]:
# General stuff
import os
import re
import time
import warnings
import math
import sys
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from subprocess import Popen, PIPE
from os.path import join as opj
from IPython.core.debugger import Tracer
from joblib import Parallel, delayed
from nilearn.decomposition import CanICA,DictLearning

# sklearn stuff
from sklearn.decomposition import TruncatedSVD,FastICA,SparsePCA,MiniBatchSparsePCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import scale
import sklearn.metrics.cluster as metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster.hierarchical import _hc_cut # Internal function to cut ward tree, helps speed up things a lot
from sklearn.utils import resample
from sklearn.model_selection import KFold

# personal functions
from importlib import reload
import fsutils as fs
reload(fs)

if os.path.exists('/data1/vbeliveau/'):
    # On NRU server
    cluster_code='/data1/vbeliveau/5HT_clustering'
    cluster_data='/data1/vbeliveau/5HT_clustering_data'
    atlas_dir='/data2/5HT_atlas'
    importPET=opj(atlas_dir,'import','PET')
    procPET=opj(atlas_dir,'proc','PET')
    subjects_dir=opj(atlas_dir,'proc','MR','recon_final')
elif os.path.exists('C:/Users/vbeliveau/'):
    # On laptop
    cluster_code='C:/Users/vbeliveau/Documents/GitHub/5HT_clustering'
    cluster_data='C:/Users/vbeliveau/Documents/5HT_clustering_data'
    subjects_dir='C:/Users/vbeliveau/Downloads/'
else:
    raise ValueError('Unknown location')
    
# Load local version of nilearn
if os.path.exists('/data1/vbeliveau/'):
    sys.path.append('/data1/vbeliveau/nilearn')
    import mynilearn.decomposition as dcm 
    reload(dcm)
elif os.path.exists('C:/Users/vbeliveau/'):
    sys.path.append('C:/Users/vbeliveau/Downloads/nilearn-master')
    import mynilearn.decomposition as dcm
    reload(dcm)
else:
    raise ValueError('Unknown location')
    
# Analysis directories
surf_data=opj(cluster_data,'surf_data')
fs.assert_dir(surf_data)
BPnd_data=opj(cluster_data,'BPnd')
fs.assert_dir(BPnd_data)
MFA_data=opj(cluster_data,'MFA')
MFA_preproc=opj(MFA_data,'preproc')
fs.assert_dir(MFA_data)
fs.assert_dir(MFA_preproc)

info_path=opj(cluster_code,'info_alltracers_base_healthy_hrrt.mat')
hemi_type=['lh','rh']

info=sio.loadmat(info_path)
tracers=['cumi','dasb','sb','az','C36']
petID=np.array([item for subl in info['petID'] for subsubl in subl for item in subsubl])
petID=petID[np.array([not re.search('^alt',x) for x in petID])]
mrID=np.array([item for subl in info['mrID'] for subsubl in subl for item in subsubl])

In [105]:
# Create cortex masks

hemi_type=['lh','rh']
targ_list=['fsaverage5','fsaverage6','fsaverage']
dest=opj(surf_data,'mask')
fs.assert_dir(dest)

for targ in targ_list:
    for hemi in hemi_type:
        fs.cortex_mask(subjects_dir,targ,hemi,validate=True,verbose=True,
                                 save_out=opj(surf_data,'mask',targ+'.'+hemi))

Reading labels
Validating cortex mask, this may take a while
Reading 327680 faces and 163842 vertices.
10000/149955
20000/149955
30000/149955
40000/149955
50000/149955
60000/149955
70000/149955
80000/149955
90000/149955
Detected invalid vertex 102161
Detected invalid vertex 102162
100000/149955
110000/149955
120000/149955
130000/149955
140000/149955
Saving masks to /data1/vbeliveau/5HT_clustering_data/surf_data/mask/fsaverage.lh
Reading labels
Validating cortex mask, this may take a while
Reading 327680 faces and 163842 vertices.
10000/149926
20000/149926
30000/149926
40000/149926
50000/149926
60000/149926
70000/149926
80000/149926
90000/149926
100000/149926
110000/149926
120000/149926
130000/149926
140000/149926
Saving masks to /data1/vbeliveau/5HT_clustering_data/surf_data/mask/fsaverage.rh
Reading labels
Validating cortex mask, this may take a while
Reading 20480 faces and 10242 vertices.
Saving masks to /data1/vbeliveau/5HT_clustering_data/surf_data/mask/fsaverage5.lh
Reading label

In [ ]:
# Create neighbor lists

hemi_type=['lh','rh']
targ_list=['fsaverage5','fsaverage6']
out_list=['matrix','list']
dest=opj(surf_data,'neigh')
fs.assert_dir(dest)

for targ in targ_list:
    for hemi in hemi_type:
        fmask=opj(surf_data,'mask',targ+'.'+hemi)
        for out_type in out_list:
            # Whole brain
            save_out=opj(surf_data,'neigh',targ+'.'+hemi)
            fs.surf_neighborhood(opj(subjects_dir,targ,'surf',hemi+'.pial'),
                                    verbose=True, out_type=out_type,save_out=save_out) 
            
            # Cortex only
            save_out=opj(surf_data,'neigh',targ+'.'+hemi+'.cortex')
            fs.surf_neighborhood(opj(subjects_dir,targ,'surf',hemi+'.pial'),mask=fmask,
                                    verbose=True, out_type=out_type,save_out=save_out)            
            

In [142]:
# Resample TACs to sphere

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
ico_list=['6']
pet_file='tac.realigned'
fwhm='5'
tac_dest=opj(surf_data,'tacs')
fs.assert_dir(tac_dest)
tac_log_dest=opj(tac_dest,'log')

for ico in ico_list:
    for hemi in hemi_type:
        def mri_vol2surf(subj):
            file = open(opj(procPET,subj,'subjectname'),'r')
            mrID=file.read()[:-1]
            file.close()
            tac_file=opj(importPET,subj,pet_file+'.nii.gz');
            reg=opj(procPET,subj,pet_file+'.wavg.GD.lta')
            out1=opj(tac_dest,subj+'.'+pet_file+'.ico'+ico+'.'+hemi+'.nii.gz')
            log_file=open(opj(tac_log_dest,'mri_vol2surf.'+subj+'.ico'+ico+'.'+hemi +'.log'),'w')
            p=Popen(['mri_vol2surf','--mov',tac_file,'--projfrac','0.5','--hemi',hemi,
                    '--o',out1,'--reg',reg,'--srcsubject',mrID,'--trgsubject','ico',
                    '--icoorder',ico],
                    stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()
        Parallel(n_jobs=20)(delayed (mri_vol2surf)(subj) for subj in petID)

In [147]:
# Smooth TACs on ico

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
ico_list=['6']
pet_file='tac.realigned'
fwhm='5'
tac_dest=opj(surf_data,'tacs')
log_dest=opj(tac_dest,'log')

for ico in ico_list:
    if ico==7:
        trg = 'fsaverage' 
    else:
        trg = 'fsaverage'+ico
    for hemi in hemi_type:
        def mris_fwhm(subj):
            out1=opj(tac_dest,subj+'.'+pet_file+'.ico'+ico+'.'+hemi+'.nii.gz')        
            out2=opj(tac_dest,subj+'.'+pet_file+'.ico'+ico+'.'+hemi+'.sm'+fwhm+'.nii.gz')
            log_file=open(opj(log_dest,'mri_surf2surf.'+subj+'.ico'+ico+'.'
                              + hemi+'.sm'+fwhm+'.log'),'w')
            p=Popen(['mris_fwhm','--i',out1,'--subject',trg,'--hemi',hemi,
                    '--cortex','--fwhm',fwhm,'--smooth-only','--o',out2],
                    stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()
        Parallel(n_jobs=20)(delayed (mris_fwhm)(subj) for subj in petID)

In [156]:
# Perform kinetic modeling of TACs on sphere

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
ico_list=['6']
pet_file='tac.realigned'
fwhm='5'
tac_dest=opj(surf_data,'tacs')
dest=opj(surf_data,'bpnd.mrtm2.nopvc')
fs.assert_dir(dest)
log_dest=opj(dest,'log')
fs.assert_dir(log_dest)

for ico in ico_list:
    if ico==7:
        trg = 'fsaverage' 
    else:
        trg = 'fsaverage'+ico
    
    for hemi in hemi_type:
        def mri_glmfit(subj):         
            ref_tac=opj(procPET,subj,'nopvc','tac.suit.cereb.dat')
            midframe=opj(procPET,subj,'midframe.sec.dat')
            file = open(opj(procPET,subj,'nopvc','mrtm-hb','k2prime.dat'),'r')
            k2p=file.read()[:-1]
            file.close()
            tac=opj(tac_dest,subj+'.'+pet_file+'.ico'+ico+'.'+hemi+'.sm'+fwhm+'.nii.gz')   
            out=opj(dest,subj+'.ico'+ico+'.'+hemi+'.sm'+fwhm)
            log_file=open(opj(log_dest,'mri_glmfit.'+subj+'.ico'+ico+'.'+ hemi +'.log'),'w')
            p=Popen(['mri_glmfit','--yhat-save','--y',tac,'--surf',trg,hemi,
                     '--o',out,'--nii.gz','--mrtm2',ref_tac,midframe,k2p],
                    stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()
        Parallel(n_jobs=20)(delayed (mri_glmfit)(subj) for subj in petID)

In [5]:
# Convert already existing BPnd maps

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
targ='fsaverage6'
sm='10'
    
convert=True
convert_mean=True

maps_dest=opj(BPnd_data,'maps')
fs.assert_dir(maps_dest)

for tracer in tracers:
    
    print(tracer)
    
    surf_dest=maps_dest + '/' + tracer    
    fs.assert_dir(surf_dest)
    
    # If BPnd was not preprocessed (e.g. fsaverage5), do it here
    if convert:        
        log_dest=opj(surf_dest,'log')
        fs.assert_dir(log_dest)
        for hemi in hemi_type:
            # Convert BPnd from fsaverage to target
            sval=opj(atlas_dir,'analyses','bmax_maps','data.nopvc.surf/', 
                tracer + '.bpnd.mrtm2.nopvc.fsaverage.' + hemi + '.sm' +  sm + '.nii.gz')
            tval=opj(surf_dest,'mrtm2.nopvc.' + targ + '.' + hemi + '.sm' + sm + '.nii.gz')

            log_file=open(opj(log_dest,'mri_surf2surf.' + hemi +'.log'),'w')
            p=Popen(['mri_surf2surf','--srcsubject','fsaverage','--trgsubject',targ,
                    '--srchemi',hemi,'--trghemi',hemi,'--sval',sval,'--tval',tval], stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()
            
    if convert_mean:
        log_dest=opj(surf_dest,'log')
        fs.assert_dir(log_dest)
        for hemi in hemi_type:
            # Convert BPnd from fsaverage to target
            sval=opj(atlas_dir,'analyses','bmax_maps','data.nopvc.surf', 
                tracer + '.mean.bpnd.mrtm2.nopvc.fsaverage.' + hemi + '.sm' +  sm + '.nii.gz')
            tval=opj(surf_dest,'mean.mrtm2.nopvc.' + targ + '.' + hemi + '.sm' + sm + '.nii.gz')

            log_file=open(opj(log_dest,'mri_surf2surf.mean.' + hemi +'.log'),'w')
            p=Popen(['mri_surf2surf','--srcsubject','fsaverage','--trgsubject',targ,
                    '--srchemi',hemi,'--trghemi',hemi,'--sval',sval,'--tval',tval], stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()

cumi
dasb
sb
az
C36


In [3]:
# Concatenate mean BPnd maps

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
targ='fsaverage5'
sm='10'

for hemi in hemi_type:
    img=np.ndarray(len(tracers),dtype=object)
    fmask=opj(surf_data,'mask.'+targ+'.'+hemi)
    for tracer,nt in zip(tracers,np.arange(0,len(tracers))):
        fname=opj(BPnd_data,'maps',tracer,'mean.mrtm2.nopvc.' + 
                  targ + '.' + hemi + '.sm' + sm + '.nii.gz')
        img[nt]=fs.load_surf_data(fname,mask=fmask)
        img[nt]=scale(img[nt],axis=0)
    fout=opj(BPnd_data,'maps','mean.scaled.concat.'+targ+'.'+hemi+'.sm'+sm+'.nii.gz')
    fs.fs_save_surf_data(np.column_stack(img),fout,mask=fmask)

AttributeError: module 'fsutils' has no attribute 'fs_load_surf_data'

In [ ]:
# Sample TACs to fsaverage surface

tracers=['cumi','dasb','sb','az','C36']
# tracers=['cumi']
hemi_type=['lh','rh']
targ='fsaverage5'
smooth=['0','5','10']

import_pet='/data1/vbeliveau/atlas/import/PET'
proc_pet='/data2/FSproc/PET'

for tracer in tracers:
    
    dest=opj(MFA_data,'surf_tacs')
    fs.assert_dir(dest)
    tracer_dest=opj(dest,tracer)
    fs.assert_dir(tracer_dest)
    log_dest=opj(tracer_dest,'log')
    fs.assert_dir(log_dest)
        
    subjlist=[item for item in petID if re.search('^'+tracer+'.*',item) is not None]
    
    for subj in subjlist:
        for hemi in hemi_type:
            for sm in smooth:

                mov=opj(import_pet,subj,'tac.realigned.nii.gz')
                reg=opj(proc_pet,subj,'tac.realigned.wavg.GD.lta')
                out=opj(tracer_dest,subj + '.' + hemi + '.' + targ + '.sm' + sm + '.nii.gz')
                cmd=['mri_vol2surf','--mov',mov,'--reg',reg,'--hemi',hemi,'--o',out,
                            '--trgsubject',targ,'--projfrac','0.5','--cortex']
                if sm != '0':
                    cmd.append('--surf-fwhm')
                    cmd.append(sm)

                log_file=open(opj(log_dest,'mri_vol2surf.' + subj + '.' + hemi + '.sm' +  sm + '.log'),'w')
                p=Popen(cmd, stdout=log_file, stderr=log_file)
                p.communicate() # This makes sure we wait for command to be executed before moving on
                log_file.close()

In [11]:
# Preprocess the raw data for MFA

tracers=['cumi','dasb','sb','az','C36']
# tracers=['cumi']
targ='fsaverage5'
smooth=['0','5','10']
hemi_type=['lh','rh'] # Note that this order is important when accessing MFA preproc data

for tracer in tracers:
    hemi_mask={}
    subjlist=[item for item in petID if re.search('^'+tracer+'.*',item) is not None]
    data_scaled=np.ndarray([len(subjlist)],dtype=object)
    for sm in smooth:
        for ns in np.arange(0,len(subjlist)):
            # Load and concatenate data
            data=None
            for hemi in hemi_type:
                fmask=opj(surf_data,'mask.'+targ+'.'+hemi)
                hemi_data=fs.load_surf_data(opj(MFA_data,'surf_tacs',tracer,subjlist[ns] + '.' +
                                 hemi + '.' + targ + '.sm' + sm + '.nii.gz'),mask=fmask)
                if data is not None:
                    data=np.vstack((data,hemi_data))
                else:
                    data=hemi_data

            # Identify empty frames in current hemisphere, and compare to other hemisphere
            frames=np.sum(data,axis=0)>10e5 # Here 10e5 is a little random, any better test?            

            # Select valid frames, remove column mean and scale by first eigen value
            data=data[:,frames]
            data=data-data.mean(axis=0)
            eig1=np.linalg.svd(data,compute_uv=False)[0]
            data_scaled[ns]=data/(eig1/1000)
            
        # Save out results
        np.savez(opj(MFA_preproc,tracer + '.' + targ + '.sm' + sm), data_scaled)